In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-large")

model.safetensors:  53%|#####2    | 1.66G/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [2]:
%pip install flask transformers torch


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, jsonify
from transformers import pipeline
import torch
import threading

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Load the translation model (Ensure this model supports translation)
translator = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# Initialize Flask
app = Flask(__name__)

@app.route('/translate', methods=['POST'])
def translate_text():
    try:
        # Get request data
        data = request.get_json()

        # Validate input
        if not data or 'text' not in data:
            return jsonify({'error': 'No text provided'}), 400

        text = data['text']
        if not isinstance(text, str) or text.strip() == "":
            return jsonify({'error': 'Invalid input. Text must be a non-empty string'}), 400

        # Perform translation
        translated_text = translator(text, max_length=512)

        return jsonify({'translation': translated_text[0]['translation_text']})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run Flask app in a separate thread
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()



The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
Press CTRL+C to quit
